# **GeneticAlgorithm**

In [16]:
import prettytable as prettytable
import random as rnd
POPULATION_SIZE = 8

class Data:
  ROOMS = [["R1", 45], ["R2", 35], ["R3", 25], ["R4", 40], ["R5", 40], ["R6", 25], ["R7", 35], ["R8", 45]]

  MEETING_TIMES = [["MT1", "MWF 09:00 - 10:00"],
                   ["MT2", "MWF 10:00 - 11:00"],
                   ["MT3", "MWF 11:00 - 12:00"],
                   ["MT4", "MWF 12:00 - 01:00"],
                   ["MT5", "TTH 09:00 - 10:30"],
                   ["MT6", "TTH 10:30 - 12:00"],
                   ["MT7", "TTH 12:00 - 01:30"],
                   ["MT8", "TTH 01:30 - 03:00"]]

  INSTRUCTORS = [["I1", "Mr.Sheikh"],
                 ["I2", "Mr.Haris"],
                 ["I3", "Mr.Khalid"],
                 ["I4", "Mr.Muhammad"],
                 ["I5", "Mr.Abdullah"],
                 ["I6", "Mr.Waseem"],
                 ["I7", "Ms.Nimra"],
                 ["I8", "Mr.Iqbal"]]
  
  def __init__(self):
    self.rooms = []; self.meetingTimes = []; self.instructors = []

    for i in range(0, len(self.ROOMS)):
        self.rooms.append(Room(self.ROOMS[i][0], self.ROOMS[i][1]))
    
    for i in range(0, len(self.MEETING_TIMES)): 
        self.meetingTimes.append(MeetingTime(self.MEETING_TIMES[i][0], self.MEETING_TIMES[i][1]))   
    
    for i in range(0, len(self.INSTRUCTORS)):
        self.instructors.append(Instructor(self.INSTRUCTORS[i][0], self.INSTRUCTORS[i][1]))

    course1 = Course("C1", "CS001", [self.instructors[0], self.instructors[1]], 25)
    course2 = Course("C2", "CS002", [self.instructors[1], self.instructors[2], self.instructors[3]], 35)
    course3 = Course("C3", "CS003", [self.instructors[0], self.instructors[1]], 25)
    course4 = Course("C4", "CS004", [self.instructors[6], self.instructors[7]], 30)
    course5 = Course("C5", "CS005", [self.instructors[3]], 35)
    course6 = Course("C6", "CS006", [self.instructors[0], self.instructors[2]], 30)
    course7 = Course("C7", "CS007", [self.instructors[4], self.instructors[5], self.instructors[6]], 35)
    course8 = Course("C8", "CS008", [self.instructors[6], self.instructors[7]], 45)
    
    self.courses = [course1, course2, course3, course4, course5, course6, course7, course8]

    dept1 = Department("BSSE", [course1, course3])
    dept2 = Department("BSCS", [course2, course4, course5])
    dept3 = Department("BBA", [course6, course7])
    dept4 = Department("MCS", [course2, course8, course5])

    self.depts = [dept1, dept2, dept3, dept4]
    self.numberOfClasses = 0
    for i in range(0, len(self.depts)):
        self.numberOfClasses += len(self.depts[i].getCourses())

  def getRooms(self):
    return self.rooms        
  def getInstructors(self):
    return self.instructors     
  def getCourses(self):
    return self.courses
  def getDepts(self):
    return self.depts
  def getMeetingTimes(self):
    return self.meetingTimes     
  def getNumberOfClasses(self):
    return self.numberOfClasses

In [17]:
class Schedule:
  def __init__(self):
    self.data = data
    self.classes = []
    self.numOfConflicts = 0
    self.fitness = -1
    self.classNum = 0
    self.isFitnessChanged = True

  def getClasses(self):
    self.isFitnessChanged = True
    return self.classes

  def getNumOfConflicts(self):
    return self.numOfConflicts

  def getFitness(self):
    if (self.isFitnessChanged == True):
        self.fitness = self.calculateFitness()
        self.isFitnessChanged == False
    return self.fitness

  def initialize(self):
    depts = self.data.getDepts()
    for i in range(0, len(depts)):
        courses = depts[i].getCourses()
        for j in range(0, len(courses)):
            newClass = Class(self.classNum, depts[i], courses[j])
            self.classNum += 1
            newClass.setMeetingTime(data.getMeetingTimes()[rnd.randrange(0, len(data.getMeetingTimes()))])
            newClass.setRoom(data.getRooms()[rnd.randrange(0, len(data.getRooms()))])
            newClass.setInstructor(courses[j].getInstructors()[rnd.randrange(0, len(courses[j].getInstructors()))])
            self.classes.append(newClass)
    return self

  def calculateFitness(self):
    self.numOfConflicts = 0
    classes = self.getClasses()
    for i in range(0, len(classes)):
        if (classes[i].getRoom().getSeatingCapacity() < classes[i].getCourse().getMaxNumOfStudents()):
            self.numOfConflicts += 1
        for j in range(0, len(classes)):
            if (j >= i):
                if (classes[i].getMeetingTime() == classes[j].getMeetingTime() and classes[i].getId() != classes[j].getId()):
                    if (classes[i].getRoom() == classes[j].getRoom()):
                      self.numOfConflicts += 1
                    if (classes[i].getInstructor() == classes[j].getInstructor()):
                      self.numOfConflicts += 1
    return 1 / ((1.0 * self.numOfConflicts + 1))

  def __str__(self):
    returnValue = ""
    for i in range(0, len(self.classes)-1):
        returnValue += str(self.classes[i]) + ", "
    returnValue += str(self.classes[len(self.classes)-1]) 
    return returnValue     

In [18]:
class Population:
  def __init__(self, size):
    self.size = size
    self.data = data
    self.schedules = []
    for i in range(0, size): 
        self.schedules.append(Schedule().initialize())

  def getSchedules(self):
    return self.schedules  

In [19]:
class GeneticAlgorithm:
  def evolve(self, population):
    return self.mutatePopulation(self.crossoverPopulation(population))
  
  def crossoverPopulation(self, pop):
    NUM_OF_ELITE_SCHEDULES = 1
    crossoverPop = Population(0)
    for i in range(NUM_OF_ELITE_SCHEDULES):
        crossoverPop.getSchedules().append(pop.getSchedules()[i])
    i = NUM_OF_ELITE_SCHEDULES
    while i < POPULATION_SIZE:
        schedule1 = self.selectTournamentPopulation(pop).getSchedules()[0]
        schedule2 = self.selectTournamentPopulation(pop).getSchedules()[0]
        crossoverPop.getSchedules().append(self.crossoverSchedule(schedule1, schedule2))
        i += 1
    return crossoverPop    

  def mutatePopulation(self, population):
    NUM_OF_ELITE_SCHEDULES = 1
    for i in range(NUM_OF_ELITE_SCHEDULES, POPULATION_SIZE):
        self.mutateSchedule(population.getSchedules()[i])
    return population    

  def crossoverSchedule(self, schedule1, schedule2):
    crossoverSchedule = Schedule().initialize()
    for i in range(0, len(crossoverSchedule.getClasses())):
        if (rnd.random() > 0.5):
            crossoverSchedule.getClasses()[i] = schedule1.getClasses()[i]
        else:
            crossoverSchedule.getClasses()[i] = schedule2.getClasses()[i]
    return crossoverSchedule        

  def mutateSchedule(self, mutateSchedule):
    MUTATION_RATE = 0.1
    schedule = Schedule().initialize()
    for i in range(0, len(mutateSchedule.getClasses())):
        if(MUTATION_RATE > rnd.random()):
            mutateSchedule.getClasses()[i] = schedule.getClasses()[i]
    return mutateSchedule    

  def selectTournamentPopulation(self, pop):
    TOURNAMENT_SELECTION_SIZE = 3
    tournamentPop = Population(0)
    i = 0
    while i < TOURNAMENT_SELECTION_SIZE:
        tournamentPop.getSchedules().append(pop.getSchedules()[rnd.randrange(0, POPULATION_SIZE)])
        i += 1
    tournamentPop.getSchedules().sort(key = lambda x: x.getFitness(), reverse = True)
    return tournamentPop

In [20]:
class Course:
  def __init__(self, number, name, instructors, maxNumOfStudents):
    self.number = number
    self.instructors = instructors
    self.name = name
    self.maxNumOfStudents = maxNumOfStudents

  def getNumber(self): 
    return self.number
  
  def getName(self):
    return self.name
  
  def getInstructors(self):
    return self.instructors
  
  def getMaxNumOfStudents(self):
    return self.maxNumOfStudents

  def __str__(self): 
    return self.name

In [21]:
class Instructor:
  def __init__(self, id, name):
    self.id = id
    self.name = name
    
  def getId(self):
    return self.id

  def getName(self):
    return self.name

  def __str__(self): 
    return self.name    

In [22]:
class Room:
  def __init__(self, number, seatingCapacity):
    self.number = number
    self.seatingCapacity = seatingCapacity

  def getNumber(self): 
    return self.number
  
  def getSeatingCapacity(self): 
    return self.seatingCapacity  

In [23]:
class MeetingTime:
  def __init__(self, id, time):
    self.id = id
    self.time = time

  def getId(self):
    return self.id
  
  def getTime(self):
    return self.time

In [24]:
class Department:
  def __init__(self, name, courses):
    self.name = name
    self.courses = courses

  def getName(self):
    return self.name
    
  def getCourses(self):
    return self.courses

In [25]:
class Class:
  def __init__(self, id, dept, course):
    self.id = id
    self.dept = dept
    self.course = course
    self.instructor = None
    self.meetingTime = None
    self.room = None

  def getId(self):
    return self.id

  def getDept(self):
    return self.dept

  def getCourse(self):
    return self.course

  def getInstructor(self):
    return self.instructor

  def getMeetingTime(self):
    return self.meetingTime

  def getRoom(self):
    return self.room

  def setInstructor(self, instructor):
    self.instructor = instructor

  def setMeetingTime(self, meetingTime):
    self.meetingTime = meetingTime

  def setRoom(self, room):
    self.room = room  

  def __str__(self):
    return str(self.dept.getName()) + "," + str(self.course.getNumber()) + "," + \
           str(self.room.getNumber()) + "," + str(self.instructor.getId()) + "," + str(self.meetingTime.getId())


In [26]:
class DisplayMgr:
  def printAvailableData(self):
    print("> All Available Data")
    self.printDept()
    self.printCourse()
    self.printRoom()
    self.printInstructor()
    self.printMeetingTimes()

  def printDept(self):
    depts = data.getDepts()
    availableDeptsTable = prettytable.PrettyTable(['dept', 'courses'])
    for i in range(0, len(depts)):
        courses = depts.__getitem__(i).getCourses()
        tempString = "["
        for j in range(0, len(courses) - 1):
            tempString += courses[j].__str__() + ", " 
        tempString += courses[len(courses) - 1].__str__() + "]"
        availableDeptsTable.add_row([depts.__getitem__(i).getName(), tempString])
    print(availableDeptsTable)

  def printCourse(self):
    availableCoursesTable = prettytable.PrettyTable(['id', 'course #', 'max # of students', 'instructors'])
    courses = data.getCourses()
    for i in range(0, len(courses)):
        instructors = courses[i].getInstructors()
        tempString = ""
        for j in range(0, len(instructors) - 1):
            tempString += instructors[j].__str__() + ", " 
        tempString += instructors[len(instructors) - 1].__str__()
        availableCoursesTable.add_row([courses[i].getNumber(), courses[i].getName(), str(courses[i].getMaxNumOfStudents()), tempString])
    print(availableCoursesTable)
  
  def printInstructor(self):
    availableInstructorsTable = prettytable.PrettyTable(['id', 'instructor'])
    instructors = data.getInstructors()
    for i in range(0, len(instructors)):
        availableInstructorsTable.add_row([instructors[i].getId(), instructors[i].getName()])
    print(availableInstructorsTable)

  def printRoom(self):
    availableRoomsTable = prettytable.PrettyTable(['room #', 'max seating capacity'])
    rooms = data.getRooms()
    for i in range(0, len(rooms)):
        availableRoomsTable.add_row([str(rooms[i].getNumber()), str(rooms[i].getSeatingCapacity())])
    print(availableRoomsTable)

  def printMeetingTimes(self):
    availableMeetingTimeTable = prettytable.PrettyTable(['id', 'Meeting Time'])
    meetingTimes = data.getMeetingTimes()
    for i in range(0, len(meetingTimes)):
        availableMeetingTimeTable.add_row([meetingTimes[i].getId(), meetingTimes[i].getTime()])
    print(availableMeetingTimeTable)    

  def printGeneration(self, population):
    table1 = prettytable.PrettyTable(['schedule #', 'fitness', '# of conflicts', 'classes [dept,class,room,instructor,meeting-time]' ])
    schedules = population.getSchedules()
    for i in range(0, len(schedules)):
        table1.add_row([str(i), round(schedules[i].getFitness(), 3), schedules[i].getNumOfConflicts(), schedules[i] ])
    print(table1)    

  def printScheduleAsTable(self, schedule):
    classes = schedule.getClasses()
    table = prettytable.PrettyTable(['Class #', 'Dept', 'Course (number, max # of students)', 'Room (Capacity)', 'Instructor (Id)', 'Meeting Time (Id)'])
    for i in range(0, len(classes)):
        table.add_row([str(i), classes[i].getDept().getName(), classes[i].getCourse().getName() + " (" +
                       classes[i].getCourse().getNumber() + ", " + str(classes[i].getCourse().getMaxNumOfStudents()) + ")",
                       classes[i].getRoom().getNumber() + " (" + str(classes[i].getRoom().getSeatingCapacity()) + ")",
                       classes[i].getInstructor().getName() + " (" + str(classes[i].getInstructor().getId()) + ")",
                       classes[i].getMeetingTime().getTime() + " (" + str(classes[i].getMeetingTime().getId()) + ")" ])
    print(table)

In [27]:
data = Data()
displyMgr = DisplayMgr()
displyMgr.printAvailableData()
generationNumber = 0
print("\n> Generation # "+str(generationNumber))
population = Population(POPULATION_SIZE)
population.getSchedules().sort(key=lambda x: x.getFitness(), reverse = True)
displyMgr.printGeneration(population)
displyMgr.printScheduleAsTable(population.getSchedules()[0])

> All Available Data
+------+-----------------------+
| dept |        courses        |
+------+-----------------------+
| BSSE |     [CS001, CS003]    |
| BSCS | [CS002, CS004, CS005] |
| BBA  |     [CS006, CS007]    |
| MCS  | [CS002, CS008, CS005] |
+------+-----------------------+
+----+----------+-------------------+----------------------------------+
| id | course # | max # of students |           instructors            |
+----+----------+-------------------+----------------------------------+
| C1 |  CS001   |         25        |       Mr.Sheikh, Mr.Haris        |
| C2 |  CS002   |         35        | Mr.Haris, Mr.Khalid, Mr.Muhammad |
| C3 |  CS003   |         25        |       Mr.Sheikh, Mr.Haris        |
| C4 |  CS004   |         30        |        Ms.Nimra, Mr.Iqbal        |
| C5 |  CS005   |         35        |           Mr.Muhammad            |
| C6 |  CS006   |         30        |       Mr.Sheikh, Mr.Khalid       |
| C7 |  CS007   |         35        | Mr.Abdullah, Mr.Wase

In [28]:
geneticAlgorithm = GeneticAlgorithm()
while (population.getSchedules()[0].getFitness() != 1.0):
    generationNumber += 1
    print("\n> Generation #" + str(generationNumber))
    population = geneticAlgorithm.evolve(population)
    population.getSchedules().sort(key = lambda x: x.getFitness(), reverse = True)
    displyMgr.printGeneration(population)
    displyMgr.printScheduleAsTable(population.getSchedules()[0])
print("\n\n")    


> Generation #1
+------------+---------+----------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| schedule # | fitness | # of conflicts |                                                                    classes [dept,class,room,instructor,meeting-time]                                                                    |
+------------+---------+----------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|     0      |  0.333  |       2        | BSSE,C1,R5,I1,MT7, BSSE,C3,R2,I2,MT5, BSCS,C2,R5,I2,MT6, BSCS,C4,R2,I7,MT3, BSCS,C5,R4,I4,MT2, BBA,C6,R6,I1,MT8, BBA,C7,R2,I5,MT7, MCS,C2,R8,I3,MT1, MCS,C8,R2,I7,MT2, MCS,C5,R5,I4,MT5 |
|     1      |  0.333  |       2        | BSSE,C1,R5,I1,MT7, BSSE,C3,R2